In [1]:
from skimage.io import imread_collection
import numpy as np
import matplotlib.pyplot as plt
import pywt as ptw
from math import floor
from sklearn.utils import shuffle
from sklearn import tree
import time
from random import randint

In [2]:
def draw(image):
    plt.imshow(image)
    plt.show()
    return

def hwavelet(images, n):
    cA = images
    for i in range(n):
        cA, (cH, cV, cD) = ptw.dwt2(cA, 'haar')
    return cA

def normalizar(dataset):
    aux = []
    for i in range(len(dataset)):
        aux = aux + [dataset[i].reshape(24*24).tolist()]
    return aux


In [3]:
# paths
pathAnger = 'images/anger/*.png'
pathContempt = 'images/contempt/*.png'
pathDisgust = 'images/disgust/*.png'
pathFear = 'images/fear/*.png'
pathHappy = 'images/happy/*.png'
pathSadness = 'images/sadness/*.png'
pathSurprise = 'images/surprise/*.png'

#images
imagesAnger = imread_collection(pathAnger)
imagesContempt = imread_collection(pathContempt)
imagesDisgust = imread_collection(pathDisgust)
imagesFear = imread_collection(pathFear)
imagesHappy = imread_collection(pathHappy)
imagesSadness = imread_collection(pathSadness)
imagesSurprise = imread_collection(pathSurprise)

# Haar Wavelet
imagesAngerHW = hwavelet(imagesAnger,1)
imagesContemptHW = hwavelet(imagesContempt,1)
imagesDisgustHW = hwavelet(imagesDisgust,1)
imagesFearHW = hwavelet(imagesFear,1)
imagesHappyHW = hwavelet(imagesHappy,1)
imagesSadnessHW = hwavelet(imagesSadness,1)
imagesSurpriseHW = hwavelet(imagesSurprise,1)

# Normalizacion
anger = normalizar(imagesAngerHW)
contempt = normalizar(imagesContemptHW)
disgust = normalizar(imagesDisgustHW)
fear = normalizar(imagesFearHW)
happy = normalizar(imagesHappyHW)
sadness = normalizar(imagesSadnessHW)
surprise = normalizar(imagesSurpriseHW)

In [4]:
x = [] # Input
y = [] # Output

for img in anger:
    x.append(img)
    y.append('ANGER')

for img in contempt:
    x.append(img)
    y.append('CONTEMPT')

for img in disgust:
    x.append(img)
    y.append('DISGUST')

for img in fear:
    x.append(img)
    y.append('FEAR')

for img in happy:
    x.append(img)
    y.append('HAPPY')

for img in sadness:
    x.append(img)
    y.append('SADNESS')

for img in surprise:
    x.append(img)
    y.append('SURPRISE')

# Shuffle data
x, y = shuffle(x, y)

In [5]:
def k_folds(x, y, k = 10):
    # indices para el testing
    inicio = 0
    fin = 0
    block = len(x) / k

    # Resultado por cada grupo (buenas y malas)
    result = []

    for i in range(k):
        # Limites del bloque de test
        inicio = block * i
        fin = block * (i + 1)

        x_testing = []
        y_testing = []
        x_training = []
        y_training = []

        # Muy lento
        for idx, punto in enumerate(x):
            if idx < inicio or idx > fin:
                x_training.append(punto)
                y_training.append(y[idx])
            else:
                x_testing.append(punto)
                y_testing.append(y[idx])

        # Crear modelo
        modelo = tree.DecisionTreeClassifier()
        modelo = modelo.fit(x_training, y_training)

        # Testing
        buenas = 0
        malas = 0
        for idx, punto in enumerate(x_testing):
            prediccion = modelo.predict([punto])
            if prediccion == y_testing[idx]:
                buenas += 1
            else:
                malas += 1
        result.append([buenas, malas])
    return result

In [6]:
def bootstrap(x, y, k = 10):
    N = len(x)
    result = []
    p = 8

    for i in range(k):
        x_testing = []
        y_testing = []
        x_training = []
        y_training = []
        indices = [0] * N # Para saber cuales estan en el training y no ponerlos en el testing
        
        # Escoger los N para el testing
        while len(x_training) < N:
            for idx, punto in enumerate(x):
                if len(x_training) >= N:
                    break
                value = randint(0, 10)
                if value <= p: # p% de posibilidades
                    x_training.append(punto)
                    y_training.append(y[idx])
                    indices[idx] = 1

        for idx, punto in enumerate(x):
            if indices[idx] == 0:
                x_testing.append(punto)
                y_testing.append(y[idx])

        # TODO: si len(x_testing) == 0: realizar otra vez el experimento
        if len(x_testing) == 0:
            continue

        # Crear modelo
        modelo = tree.DecisionTreeClassifier()
        modelo = modelo.fit(x_training, y_training)

        # Testing
        buenas = 0
        malas = 0
        for idx, punto in enumerate(x_testing):
            prediccion = modelo.predict([punto])
            if prediccion == y_testing[idx]:
                buenas += 1
            else:
                malas += 1
        result.append([buenas, malas])
    return result

In [7]:
k = 10

start = time.time()
result_k_folds = k_folds(x, y, k)
end = time.time()
print('Tiempo total de testing con', 'k-folds (k = ' + str(k) + ')', end-start, 'segundos.')
i = 1
for buenas, malas in result_k_folds:
    print('Experimento', i)
    print('\t', 'Buenas: ' + str(buenas) + '. Malas: ' + str(malas))
    i += 1


start = time.time()
result_bootstrap = bootstrap(x, y, k)
end = time.time()
print('\n\nTiempo total de testing con bootstrap (' + str(k) + ' experimentos)', end-start, 'segundos.')
i = 1
for buenas, malas in result_bootstrap:
    print('Experimento', i)
    print('\t', 'Buenas: ' + str(buenas) + '. Malas: ' + str(malas))
    i += 1

Tiempo total de testing con k-folds (k = 10) 3.125 segundos.
Experimento 1
	 Buenas: 84. Malas: 15
Experimento 2
	 Buenas: 87. Malas: 11
Experimento 3
	 Buenas: 85. Malas: 13
Experimento 4
	 Buenas: 88. Malas: 10
Experimento 5
	 Buenas: 82. Malas: 16
Experimento 6
	 Buenas: 89. Malas: 9
Experimento 7
	 Buenas: 83. Malas: 15
Experimento 8
	 Buenas: 86. Malas: 12
Experimento 9
	 Buenas: 81. Malas: 17
Experimento 10
	 Buenas: 90. Malas: 8


Tiempo total de testing con bootstrap (10 experimentos) 3.3129982948303223 segundos.
Experimento 1
	 Buenas: 120. Malas: 22
Experimento 2
	 Buenas: 133. Malas: 20
Experimento 3
	 Buenas: 132. Malas: 20
Experimento 4
	 Buenas: 132. Malas: 18
Experimento 5
	 Buenas: 121. Malas: 23
Experimento 6
	 Buenas: 118. Malas: 15
Experimento 7
	 Buenas: 117. Malas: 17
Experimento 8
	 Buenas: 117. Malas: 11
Experimento 9
	 Buenas: 121. Malas: 15
Experimento 10
	 Buenas: 139. Malas: 17
